<a href="https://colab.research.google.com/github/ArtyomShabunin/SMOPA-25/blob/main/lesson_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://prana-system.com/files/110/rds_color_full.png" alt="tot image" width="300"  align="center"/> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src="https://mpei.ru/AboutUniverse/OficialInfo/Attributes/PublishingImages/logo1.jpg" alt="mpei image" width="200" align="center"/>
<img src="https://mpei.ru/Structure/Universe/tanpe/structure/tfhe/PublishingImages/tot.png" alt="tot image" width="100"  align="center"/>

---

# **Системы машинного обучения и предиктивной аналитики в тепловой и возобновляемой энергетике**  

# ***Практические занятия***


---



# Занятие №4
# Поиск аномалий методами глубокого обучения
**12 марта 2025г.**

In [ ]:
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import seaborn as sns
sns.set_theme(style="whitegrid", rc={'figure.figsize':(15,6)})

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F

from tqdm import tqdm
import json

import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.graph_objs as go

pd.options.mode.chained_assignment = None

from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

## Загрузка данных

In [ ]:
# import gdown
# import warnings
# warnings.filterwarnings('ignore')
# gdown.download('https://drive.google.com/uc?id=1uEm4d4rbh1RAPw4JOHJiHzw5y_m3OSj3', verify=False)

data = pd.read_parquet("./data.gzip")

# signals = [
#     'GTA1.DBinPU.Aldi', 'GTA1.DBinPU.Alvna', 'GTA1.DBinPU.Alzzo',
#     'GTA1.DBinPU.Bo', 'GTA1.DBinPU.fi', 'GTA1.DBinPU.nst',
#     'GTA1.DBinPU.ntk', 'GTA1.DBinPU.P', 'GTA1.DBinPU.Pk', 'GTA1.DBinPU.Pvh',
#     'GTA1.DBinPU.Qtg', 'GTA1.DBinPU.Tk', 'GTA1.DBinPU.Tn', 'GTA1.DBinPU.Tt']

# data = data.loc[:, signals]

In [ ]:
data.head()

Чтение файла с описанием сигналов

In [ ]:
with open(f'./option_0/description.json', 'r', encoding = "utf-8") as f:
    description = json.load(f)

Составим словарь для трактовки наименований сигналов

In [ ]:
kks_to_description = {param['real_kks']: f"{param['description']}, [{param['unit']}]"
for param in description if param['real_kks'] in data.columns}

description_to_kks = { f"{param['description']}, [{param['unit']}]": param['real_kks']
for param in description if param['real_kks'] in data.columns}

### Деление на тренировочную и тестовую выборки

In [ ]:
shuffled_data = data.sample(frac=1)

data_train = shuffled_data.iloc[:round(shuffled_data.shape[0]*0.8), :]
data_test = shuffled_data.iloc[round(shuffled_data.shape[0]*0.8):, :]

Добавим шум на половину тестовых данных и отметим их как аномальные

In [ ]:
def generate_binary_matrix(rows, cols, prob_ones=0.4):
    matrix = np.zeros((rows, cols), dtype=int)  # Создаем матрицу из нулей

    for i in range(rows):
        # Генерируем случайные 0 и 1 с заданной вероятностью (без гарантированной 1)
        row = np.random.choice([0, 1], size=cols, p=[1 - prob_ones, prob_ones])

        # Если в строке нет 1, вставляем её в случайное место
        if not np.any(row):
            row[np.random.randint(0, cols)] = 1

        matrix[i] = row  # Записываем строку в матрицу

    return matrix

In [ ]:
NUMBER_OF_ANOMALY_POINTS = round(data_test.shape[0]*0.5)

data_test["anomaly"] = 0
data_test.loc[
    data_test.iloc[:NUMBER_OF_ANOMALY_POINTS].index, ["anomaly"]] = 1

mask = generate_binary_matrix(NUMBER_OF_ANOMALY_POINTS, data.shape[1], prob_ones=0.3)

bias = mask * np.random.choice(
    [-0.05, -0.04, -0.03, -0.02, -0.01, 0.01, 0.02, 0.03, 0.04, 0.05],
    size=[NUMBER_OF_ANOMALY_POINTS, data.shape[1]],
    p=np.full(10,0.1))

data_test.loc[
    data_test.iloc[:NUMBER_OF_ANOMALY_POINTS].index,
    data_test.columns != "anomaly"] = data_test[data_test["anomaly"] == 1].loc[:,data_test.columns != "anomaly"] * (1 + bias)

In [ ]:
params_dropdown = widgets.Dropdown(
    options=data_test.columns,
    description='Параметр:',
    disabled=False,
    value=None
)

out = widgets.Output()
display(out)

with out:
    display(params_dropdown)

@out.capture()
def params_dropdown_eventhandler(change):

    clear_output()
    display(params_dropdown)

    fig, axes = plt.subplots(1, 1, figsize=(15,5))
    plt.title(f"Тестовая выборка\n{change.new} - {kks_to_description [change.new]}")

    data_test[data_test["anomaly"] == 0][change.new].plot(style='.', label="Нормальные данные");
    data_test[data_test["anomaly"] == 1][change.new].plot(style='.', label="Зашумленные данные");

    # plt.plot(scat_1[change.new], 'r.', markersize=5, label='Аномалии')
    # plt.plot(scat_0[change.new], 'g.', markersize=5, label='Норма')
    plt.legend()
    display(fig)

params_dropdown.observe(params_dropdown_eventhandler, names='value')

### Нормализация или стандартизация данных

In [ ]:
scaler = preprocessing.MinMaxScaler() # нормализация данных
# scaler = preprocessing.StandardScaler() # стандартизация данных

X_train = pd.DataFrame(
    scaler.fit_transform(data_train),
    columns=data_train.columns,
    index=data_train.index)

X_test = pd.DataFrame(
    scaler.transform(data_test[data.columns]),
    columns=data.columns,
    index=data_test.index)

X_train.describe()

## Полносвязная нейронная сеть (Fully Connected Layer, FC Layer)

Полносвязный слой — это основной компонент нейронных сетей, где каждый нейрон текущего слоя соединён со всеми нейронами предыдущего и следующего слоя.

Персептрон — это **базовая модель** искусственной нейронной сети, предложенная Фрэнком Розенблаттом в 1958 году. Это **самый простой вид нейросети**, который является основой для современных **полносвязных нейронных сетей**.

---
### **Персептрон**
**Структура персептрона**

<img src="https://neerc.ifmo.ru/wiki/images/a/a5/%D0%98%D1%81%D0%BA%D1%83%D1%81%D1%81%D1%82%D0%B2%D0%B5%D0%BD%D0%BD%D1%8B%D0%B9_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D0%BD_%D1%81%D1%85%D0%B5%D0%BC%D0%B0.png" alt="perceptron image" width="500"/>

Классический **одиночный персептрон** состоит из:  
- **Входного слоя** $ x_1, x_2, ..., x_n $ — входные признаки  
- **Весов** $ w_1, w_2, ..., w_n $ — коэффициенты, определяющие важность входных признаков  
- **Смещения** (bias) $ b $ — позволяет модели лучше подстраиваться  
- **Сумматора** — вычисляет взвешенную сумму входов:  
  $ z = w_1 x_1 + w_2 x_2 + ... + w_n x_n + b $
- **Активационной функции** $ f(z) $, например, пороговой (step function):  
  $$
  y =
  \begin{cases}
  1, & z \geq 0 \\
  0, & z < 0
  \end{cases}
  $$
  
Персептрон работает как **линейный классификатор**, разделяя данные на два класса.

---
### **Многослойный персептрон**

- **Одиночный персептрон** — это всего **один полносвязный слой**.  
- **Многослойный персептрон (MLP, Multi-Layer Perceptron)** — это **полносвязная нейросеть**, состоящая из **нескольких персептронов** в скрытых слоях.  
- В отличие от обычного персептрона, **MLP может моделировать нелинейные зависимости** благодаря использованию **нелинейных активационных функций** (ReLU, Sigmoid, Tanh).

<img src="https://lh5.googleusercontent.com/proxy/_NrlkWwft7zy_-C-fxPidblqeLH0wZXd5Vj2MzsM0twc0mp72IPtSrpijoQ-4rfx9BSBbHbFAkXo7BaMXKdAaUhg2tGH" alt="mlp image" width="400"/>

---

**Формула двухслойного персептрона через матричное умножение**  

Рассмотрим **двухслойный персептрон** (один скрытый слой + выходной слой).  

**Обозначения:**  
- $ X $ — входной вектор (размерность $ (n, d) $, где $ n $ — количество примеров, $ d $ — число входных признаков).  
- $ W_1 $ — матрица весов первого слоя (размерность $ (d, h) $, где $ h $ — число нейронов в скрытом слое).  
- $ b_1 $ — вектор смещений первого слоя (размерность $ (1, h) $).  
- $ W_2 $ — матрица весов второго слоя (размерность $ (h, k) $, где $ k $ — число выходных нейронов).  
- $ b_2 $ — вектор смещений второго слоя (размерность $ (1, k) $).  
- $ f $ — активационная функция (например, ReLU или сигмоида).  
- $ g $ — активация выходного слоя (например, Softmax для классификации).  

---

**1. Вычисление скрытого слоя**  
$$ H = f(X W_1 + b_1) $$
- $ X $ имеет размерность $ (n, d) $, $ W_1 $ — $ (d, h) $, $ b_1 $ — $ (1, h) $.  
- После матричного умножения $ X W_1 $ получаем размерность $ (n, h) $.  
- Добавляем $ b_1 $ и применяем функцию активации $ f $, получая матрицу активаций скрытого слоя $ H $ размерности $ (n, h) $.  

**2. Вычисление выходного слоя**  
$$ Y = g(H W_2 + b_2) $$
- $ H $ имеет размерность $ (n, h) $, $ W_2 $ — $ (h, k) $, $ b_2 $ — $ (1, k) $.  
- После умножения $ H W_2 $ получаем размерность $ (n, k) $.  
- Добавляем $ b_2 $ и применяем активацию $ g $, получая выходной вектор $ Y $ размерности $ (n, k) $.  

---

**Итоговая запись**  
$$ Y = g(f(X W_1 + b_1) W_2 + b_2) $$

---

### **Функция активации ReLU (Rectified Linear Unit)**  

**ReLU (Rectified Linear Unit)** — одна из самых популярных активационных функций в нейронных сетях. Она используется в скрытых слоях, чтобы добавлять нелинейность в модель.

---

**Формула ReLU**  
$$ f(x) = \max(0, x) $$
- Если $ x > 0 $, то $ f(x) = x $  
- Если $ x \leq 0 $, то $ f(x) = 0 $  

**ReLU обнуляет отрицательные значения и пропускает положительные без изменений.**

---

**ReLU — стандарт для глубоких нейросетей**.  

## Автокодировщик (autoencoder)  

Автокодировщик (**Autoencoder, AE**) — это нейросеть, обучаемая на нормальных данных, чтобы восстанавливать их с минимальной ошибкой. Аномалии выявляются по значению ошибки восстановления (**reconstruction error**): если ошибка высокая, то входные данные, скорее всего, аномальные.

<!-- <img src="https://pub.mdpi-res.com/information/information-12-00238/article_deploy/html/images/information-12-00238-g001.png" alt="ae image" width="500"/> -->

<img src="https://upload.wikimedia.org/wikipedia/commons/3/37/Autoencoder_schema.png" alt="ae image" width="500"/>

---

### **1. Кодировщик (Encoder)**
Сжимает входные данные в скрытое представление (**latent space**) меньшей размерности.  

Пример:  
$$ h = f(Wx + b) $$
Где $ f $ — нелинейная активация (ReLU, LeakyReLU), \( W \) — веса сети.

### **2. Боттлнек (Latent Space)**
Самое узкое место в сети, где модель сохраняет только ключевые признаки данных.

### **3. Декодировщик (Decoder)**
Восстанавливает данные из латентного представления обратно в исходное пространство.

Пример:  
$$ \hat{x} = g(W'h + b') $$

### **4. Вычисление ошибки восстановления**
- **Mean Squared Error (MSE)**:  
  $$ \text{Loss} = \frac{1}{n} \sum (x - \hat{x})^2 $$
- Если ошибка выше порога, то это **аномалия**.

---

**Настройки обучения**
- **Функция активации**: ReLU или LeakyReLU в кодировщике, сигмоида в выходном слое (если входные данные нормализованы).
- **Функция потерь**: MSE.
- **Оптимизатор**: Adam.
- **Регуляризация**: Dropout или L1/L2-регуляризация, чтобы улучшить генерализацию.

---

## **Сравнение с One-Class SVM**
| **Критерий**       | **Автокодировщик (AE)** | **One-Class SVM (OC-SVM)** |
|--------------------|------------------------|------------------------|
| **Тип модели** | Нейросеть с обучением | Метод SVM |
| **Пространство признаков** | Латентное представление | Гиперплоскость |
| **Адаптивность** | Гибкая, можно дообучать | Фиксированные границы |
| **Работа с нелинейностями** | Легко захватывает сложные структуры | Использует ядра, но ограничено |
| **Выявление аномалий** | По ошибке восстановления | По расстоянию от гиперплоскости |
| **Требования к данным** | Требуется большое количество нормальных данных | Работает даже с малым объемом |
| **Объясняемость** | Трудно интерпретировать | Легче понять границы класса |

---

- **AE лучше**, если аномалии имеют **сложные структуры** и нужно анализировать нелинейные зависимости.
- **OC-SVM лучше**, если данных **мало** и аномалии — это просто редкие отклонения.

Если данные промышленные и содержат **много параметров**, то **автокодировщик** обычно показывает **лучшие результаты**.


## Построение модели

In [ ]:
features = data.shape[1]

class Autoencoder(torch.nn.Module):
    def __init__(self, h):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(features, h),
            nn.ReLU(),
            nn.Linear(h, h),
        )
        self.decoder = nn.Sequential(
            nn.Linear(h, features),
            nn.ReLU(),
            nn.Linear(features, features),
        )

    def forward(self, sample):
        latent = self.encoder(sample)
        reconstructed = self.decoder(latent)
        return reconstructed

In [ ]:
model = Autoencoder(5)
model

In [ ]:
x = torch.tensor(X_train.iloc[10].values, dtype=torch.float32)[np.newaxis,:]
x_hat = model(x)

In [ ]:
x

In [ ]:
x_hat

## **`torch.tensor` в PyTorch**  

В **PyTorch** `torch.tensor` — это **основная структура данных** для работы с числовыми массивами (аналог `numpy.ndarray`), которая поддерживает вычисления на **GPU** и **автоматическое дифференцирование**.

Создание тензора из списка

In [ ]:
x = torch.tensor([1, 2, 3])
x

In [ ]:
x.dtype

Задание типа данных (`dtype`)

In [ ]:
x = torch.tensor([1, 2, 3], dtype=torch.float32)
x.dtype

In [ ]:
x.to(torch.int32)

Создание тензора с нулями и единицами

In [ ]:
zeros = torch.zeros(3, 3)  # Матрица 3x3 из нулей
ones = torch.ones(2, 2)    # Матрица 2x2 из единиц
rand = torch.rand(2, 3)    # Матрица 2x3 со случайными числами

zeros, ones, rand

Основные свойства тензоров

In [ ]:
x = torch.tensor([[1, 2, 3], [4, 5, 6]])

print(x.shape)  # Размерность (2, 3)
print(x.dtype)  # Тип данных (int64)
print(x.device) # Где хранится (CPU/GPU)

Перевод на GPU

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
x = torch.tensor([1, 2, 3], device=device)
print(x.device)  # cuda (если есть GPU)

Преобразование между NumPy и PyTorch

In [ ]:
# Из NumPy в PyTorch
np_array = np.array([1, 2, 3])
torch_tensor = torch.from_numpy(np_array)

# Из PyTorch в NumPy
numpy_array = torch_tensor.numpy()

In [ ]:
torch_tensor = torch.from_numpy(np_array)
torch_tensor.detach().numpy() # detach() используется в PyTorch, чтобы отключить градиенты у тензора.
                              # Это позволяет создать новый тензор, который не участвует в вычислении градиентов,
                              # но содержит те же значения, что и исходный тензор

Операции с тензорами

In [ ]:
x = torch.tensor([1, 2, 3])
y = torch.tensor([4, 5, 6])

print(x + y)  # Сложение
print(x * y)  # Поэлементное умножение
print(x @ y)  # Скалярное произведение

Автоматическое вычисление градиентов (`requires_grad=True`)

In [ ]:
x = torch.tensor([2.0], requires_grad=True)
y = x**2  # y = x^2

y.backward()  # Вычисляем градиент
print(x.grad)  # dy/dx = 2*x = 4

Добавление размерности

In [ ]:
x = torch.tensor([1, 2, 3])  # Размерность: (3,)
x_unsqueezed = x.unsqueeze(0)  # Добавляем ось по нулевому измерению
print(x_unsqueezed.shape)  # torch.Size([1, 3])

In [ ]:
x = torch.tensor([1, 2, 3])  # Размерность: (3,)
x_reshaped = x.view(1, 3)  # То же самое, что unsqueeze(0)
print(x_reshaped.shape)  # torch.Size([1, 3])

In [ ]:
# индексация
x = torch.tensor([1, 2, 3])  # Размерность: (3,)
x_expanded = x[None, :]
print(x_expanded.shape)  # torch.Size([1, 3])

Удаление размерности

In [ ]:
x = torch.tensor([[1, 2, 3]])  # Размерность (1, 3)
x_squeezed = x.squeeze(0)  # Удаляем нулевую ось
print(x_squeezed.shape)  # torch.Size([3])

In [ ]:
# индексация
x = torch.tensor([[1, 2, 3]])  # Размерность (1, 3)
x_squeezed = x_expanded[0]
print(x_squeezed.shape)  # torch.Size([3])

## Обучение сети

**Прямой проход (Forward Pass)**  
- Данные проходят через сеть, и каждый слой применяет свои веса и функции активации.  
- Получаем **выход модели** (предсказание).  

**Вычисление ошибки (Loss Calculation)**  
- Считаем **функцию потерь** (например, MSE, CrossEntropy).  
- Показывает, насколько сильно предсказание отличается от правильного ответа.

Среднеквадратичная ошибка (MSE) вычисляется как **среднее арифметическое квадратов разностей между предсказанными ($\hat{y}_i$) и реальными ($y_i$) значениями**:  

$$MSE = \frac{1}{N} \sum_{i=1}^{N} (y_i - \hat{y}_i)^2$$

Где:  
- $ N $ — количество примеров в выборке  
- $ y_i $ — истинное значение  
- $ \hat{y}_i $ — предсказанное значение модели  

---

**Обратное распространение ошибки (Backward Pass)**  
- Вычисляем **градиенты** ошибки по всем весам сети, начиная с выхода.  
- Используем **цепное правило дифференцирования** (chain rule), чтобы передавать градиенты назад по сети.  

**Обновление весов (Weight Update)**  
- Используем **оптимизатор** (например, `SGD`, `Adam`), чтобы обновить веса:  
  $$ w = w - \eta \cdot \frac{\partial L}{\partial w} $$
  где:  
  - $ w $ — веса сети  
  - $ \eta $ — скорость обучения (learning rate)  
  - $ \frac{\partial L}{\partial w} $ — градиент ошибки  


Заметим, что поскольку мы занимаемся реконструкцией, train/val у нас выступает как в роли входа для сети, так и в роли таргета

### Создаем `DataLoader` на обучающих и тестовых данных

Делим нашу обучающую выборку на обучающую и валидационную

In [ ]:
X_val = X_train.iloc[round(X_train.shape[0]*0.8):, :]
X_train = X_train.iloc[:round(X_train.shape[0]*0.8), :]

In [ ]:
BATCH_SIZE = 512 # Размер батча (количество примеров за раз)
# Разделение данных на батчи (mini-batches) при обучении нейросетей ускоряет и стабилизирует процесс градиентного спуска.

train_loader = torch.utils.data.DataLoader(X_train.values, batch_size=BATCH_SIZE)
val_loader = torch.utils.data.DataLoader(X_val.values, batch_size=BATCH_SIZE)

### Обучение модели

In [ ]:
HIDDEN_SIZE = 10
N_EPOCHS = 100
LEARNING_RATE = 0.001


model = Autoencoder(HIDDEN_SIZE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss_fn = F.mse_loss
train_losses = []
val_losses = []

for epoch in tqdm(range(N_EPOCHS)):
    model.train()
    train_losses_per_epoch = []
    for i, X_batch in enumerate(train_loader):
        X_batch = X_batch.to(torch.float32)
        optimizer.zero_grad()
        reconstructed = model(X_batch)
        loss = loss_fn(reconstructed, X_batch)
        loss.backward()
        optimizer.step()
        train_losses_per_epoch.append(loss.item())

    train_losses.append(np.mean(train_losses_per_epoch))

    model.eval()
    val_losses_per_epoch = []
    with torch.no_grad():
        for X_batch in val_loader:
            X_batch = X_batch.to(torch.float32)
            reconstructed = model(X_batch)
            loss = loss_fn(reconstructed, X_batch)
            val_losses_per_epoch.append(loss.item())

    val_losses.append(np.mean(val_losses_per_epoch))

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(np.arange(len(train_losses)), train_losses, label='Train')
plt.plot(np.arange(len(val_losses)), val_losses, label='Validation')

plt.xlabel('Эпоха')
plt.title('Среднеквадратичная ошибка (MSE loss)')
plt.legend()
plt.show()

In [ ]:
print('Loss в конце обучения')
print(f'На обучающей выборке: {train_losses[-1]:.5f}')
print(f'На валидационной выборке: {val_losses[-1]:.5f}')

## Оценка ошибки восстановления

In [ ]:
x = torch.tensor(X_test.values, dtype=torch.float32)
x_hat = model(x)
reconstruction_err = F.mse_loss(x_hat, x, reduction='none').mean(axis=1)
data_test['reconstruction_err'] = reconstruction_err.detach().numpy()

In [ ]:
try:
    scat_1 = data_test.groupby('anomaly_svm').get_group(1)
    scat_0 = data_test.groupby('anomaly_svm').get_group(0)
except KeyError:
    print("Не удалось разделить данные")

params_dropdown = widgets.Dropdown(
    options=data_test.columns,
    description='Параметр:',
    disabled=False,
    value=None
)

out = widgets.Output()
display(out)

with out:
    display(params_dropdown)

@out.capture()
def params_dropdown_eventhandler(change):

    clear_output()
    display(params_dropdown)
    # selected_param = selected_params_kks[list(selected_params_description).index(change.new)]


    fig, axes = plt.subplots(1, 1, figsize=(15,5))
    fig = plt.figure();



    plt.scatter(
        data_test[change.new].index,
        data_test[change.new].values,
        c = data_test['reconstruction_err'],
        s=10, alpha=0.5);


    plt.title(f"{change.new} - {kks_to_description [change.new]}")
    plt.ylabel(change.new, fontsize=20);
    cbar = plt.colorbar()
    plt.set_cmap('viridis')
    cbar.ax.get_yaxis().labelpad = 20
    cbar.ax.set_ylabel('reconstruction_err', rotation=270, fontsize=20)
    display(fig)

params_dropdown.observe(params_dropdown_eventhandler, names='value')

### **ROC AUC — способность модели различать классы**
ROC AUC показывает **насколько хорошо модель различает нормальные и аномальные данные**, независимо от конкретного порога.  

**Как строится?**  
- **ROC-кривая** — график зависимости **True Positive Rate (TPR)** от **False Positive Rate (FPR)** при разных значениях порога.
- **AUC (Area Under Curve)** — площадь под этой кривой.

$$ TPR = \frac{TP}{TP + FN}, \quad FPR = \frac{FP}{FP + TN}$$

**Как интерпретировать?**  
✔ **AUC = 0.5** → модель не лучше случайного угадывания  
✔ **AUC → 1.0** → модель отлично различает аномалии и нормальные данные  
✔ **AUC < 0.5** → модель ошибочно классифицирует нормальные данные как аномалии  

**Важно:**  
- ROC AUC **не зависит от конкретного порога**, в отличие от F1-score.  
- Если классы сильно **дисбалансированы**, ROC AUC может быть **обманчиво высокой** (например, если аномалий мало, модель может просто предсказывать «норму» и получить высокий AUC).


In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc = roc_auc_score(data_test['anomaly'], data_test['reconstruction_err'])
print(f"ROC AUC: {roc_auc:.4f}")

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(data_test['anomaly'], data_test['reconstruction_err'])

plt.figure(figsize=(15, 5))
plt.plot(fpr, tpr, color='blue', label=f'ROC curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')  # Линия случайного угадывания
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Anomaly Detection')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

### Оптимальный порог

In [ ]:
# Рассчитываем расстояние от точки (0, 1)
distances = np.sqrt(fpr**2 + (1 - tpr)**2)

# Находим индекс минимального расстояния
optimal_threshold_index = np.argmin(distances)

# Оптимальный порог
optimal_threshold = thresholds[optimal_threshold_index]
print(f"Оптимальный порог: {optimal_threshold:.4f}")

### Accuracy, Precision, Recall, F1

In [ ]:
data_test['anomaly_ae'] = (data_test['reconstruction_err'] > optimal_threshold).astype(int)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

print(f"accuracy - {accuracy_score(data_test['anomaly'], data_test['anomaly_ae'])*100:0.2f}%")
print(f"precision - {precision_score(data_test['anomaly'], data_test['anomaly_ae'], average='binary')*100:0.2f}%")
print(f"recall - {recall_score(data_test['anomaly'], data_test['anomaly_ae'], average='binary')*100:0.2f}%")
print(f"F1 - {f1_score(data_test['anomaly'], data_test['anomaly_ae'], average='binary')*100:0.2f}%")